In [ ]:
# Step 1: Install libraries
!pip install wikipedia-api neo4j

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.6/296.6 kB 5.2 MB/s eta 0:00:00
  Created wheel for wikipedia-api: filename=Wikipedia_API-0.7.1-py3-none-any.whl size=14346 sha256=ea8ad489e7d5bb03ff0826b1e8b525d5da3dc296b46217a4998330d0b4a36150
  Stored in directory: /root/.cache/pip/wheels/4c/96/18/b9201cc3e8b47b02b510460210cfd832ccf10c0c4dd0522962
Successfully built wikipedia-api


In [ ]:
!pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.7 MB/s eta 0:00:00


In [ ]:
# Step 2: Import libraries
import wikipediaapi
import openai
from neo4j import GraphDatabase
import re

In [ ]:
# Step 3: Set OpenAI API Key for GPT-based LLM
openai.api_key = 'sk-r7L1feU_nIMoF8sLmKCUqKlE9X3PLShebgyJUUaI1BT3BlbkFJCWoi7KHB_oiwLcQor1F6QipfdM_RWEB6gjCHW1cRkA'

In [ ]:
# Function to extract entities from text using OpenAI GPT LLM
def extract_entities(text):
    prompt = f"Extract entities and their types from the following text:\n\n{text}\n\nFormat: Entity - Type (e.g., John Doe - Person)"
    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}]
    )
    return response['choices'][0]['message']['content']

In [ ]:
# Step 4: Connect to Neo4j Sandbox Cloud
uri = "bolt://44.204.200.141:7687"
username = "neo4j"
password = "raincoats-wheel-assemblies"
driver = GraphDatabase.driver(uri, auth=(username, password))

In [ ]:
# Function to create nodes in Neo4j
def create_node(tx, label, name):
    query = f"MERGE (n:{label} {{name: $name}})"
    tx.run(query, name=name)

In [ ]:
# Function to create relationships in Neo4j
def create_relationship(tx, entity1, entity2, relationship):
    query = (
        f"MATCH (a {{name: $entity1}}), (b {{name: $entity2}}) "
        f"MERGE (a)-[r:{relationship}]->(b) "
        "RETURN type(r)"
    )
    tx.run(query, entity1=entity1, entity2=entity2)

In [ ]:
import wikipediaapi

# Step 5: Fetch Wikipedia page data
wiki = wikipediaapi.Wikipedia(
    language='en',
    user_agent='my_bot (abc@example.com)'
)
page_title = "Artificial_intelligence"
page = wiki.page(page_title)

if page.exists():
    wiki_text = page.text
    print(f"Text from {page_title}: \n", wiki_text[:1000])  # Preview first 1000 characters
else:
    print(f"The page '{page_title}' does not exist.")

Text from Artificial_intelligence: 
 Artificial intelligence (AI), in its broadest sense, is intelligence exhibited by machines, particularly computer systems. It is a field of research in computer science that develops and studies methods and software that enable machines to perceive their environment and use learning and intelligence to take actions that maximize their chances of achieving defined goals. Such machines may be called AIs.
Some high-profile applications of AI include advanced web search engines (e.g., Google Search); recommendation systems (used by YouTube, Amazon, and Netflix); interacting via human speech (e.g., Google Assistant, Siri, and Alexa); autonomous vehicles (e.g., Waymo); generative and creative tools (e.g., ChatGPT, and AI art); and superhuman play and analysis in strategy games (e.g., chess and Go). However, many AI applications are not perceived as AI: "A lot of cutting edge AI has filtered into general applications, often without being called AI because 

In [ ]:
# Step 6: Extract entities from Wikipedia text
entities_text = extract_entities(wiki_text[:2000])  # Process portion of the text for demo
print("Extracted Entities:\n", entities_text)

Extracted Entities:
 1. Artificial intelligence - Concept
2. AI - Concept
3. machines - Concept
4. computer systems - Concept
5. computer science - Field
6. Google Search - Application
7. YouTube - Application
8. Amazon - Application
9. Netflix - Application
10. Google Assistant - Application
11. Siri - Application
12. Alexa - Application
13. Waymo - Application
14. ChatGPT - Application
15. AI art - Application
16. chess - Game
17. Go - Game
18. reasoning - Goal
19. knowledge representation - Goal
20. planning - Goal
21. learning - Goal
22. natural language processing - Goal
23. perception - Goal
24. support for robotics - Goal
25. General intelligence - Concept
26. artificial neural networks - Technique
27. statistics - Field
28. operations research - Field
29. economics - Field
30. psychology - Field
31. linguistics - Field
32. philosophy - Field
33. neuroscience - Field
34. 1956 - Year


In [ ]:
# Step 7: Parse and create relationships
import re

# Function to remove leading numbers, periods, and spaces
def clean_entity_name(entity_name):
    return re.sub(r"^\d+\.\s*", "", entity_name).strip()

def parse_entities(entities_text):
    entity_pattern = r"(.+) - (.+)"
    entities = []
    for line in entities_text.split('\n'):
        match = re.match(entity_pattern, line)
        if match:
            entity_name, entity_type = match.groups()
            entity_name = clean_entity_name(entity_name)
            entities.append((entity_name, entity_type.strip()))
    return entities

entities = parse_entities(entities_text)

In [ ]:
entities

[('Artificial intelligence', 'Concept'),
 ('AI', 'Concept'),
 ('machines', 'Concept'),
 ('computer systems', 'Concept'),
 ('computer science', 'Field'),
 ('Google Search', 'Application'),
 ('YouTube', 'Application'),
 ('Amazon', 'Application'),
 ('Netflix', 'Application'),
 ('Google Assistant', 'Application'),
 ('Siri', 'Application'),
 ('Alexa', 'Application'),
 ('Waymo', 'Application'),
 ('ChatGPT', 'Application'),
 ('AI art', 'Application'),
 ('chess', 'Game'),
 ('Go', 'Game'),
 ('reasoning', 'Goal'),
 ('knowledge representation', 'Goal'),
 ('planning', 'Goal'),
 ('learning', 'Goal'),
 ('natural language processing', 'Goal'),
 ('perception', 'Goal'),
 ('support for robotics', 'Goal'),
 ('General intelligence', 'Concept'),
 ('artificial neural networks', 'Technique'),
 ('statistics', 'Field'),
 ('operations research', 'Field'),
 ('economics', 'Field'),
 ('psychology', 'Field'),
 ('linguistics', 'Field'),
 ('philosophy', 'Field'),
 ('neuroscience', 'Field'),
 ('1956', 'Year')]

In [ ]:
# Create nodes and relationships in Neo4j
with driver.session() as session:
    for i, (entity1_name, entity1_type) in enumerate(entities):
        # Create nodes for the first entity
        session.write_transaction(create_node, entity1_type, entity1_name)

        for j, (entity2_name, entity2_type) in enumerate(entities):
            if i != j:
                # Define relationships based on entity types
                if entity1_type == "Application" and entity2_type == "Concept":
                    session.write_transaction(create_relationship, entity1_name, entity2_name, "USES")
                elif entity1_type == "Application" and entity2_type == "Goal":
                    session.write_transaction(create_relationship, entity1_name, entity2_name, "ACHIEVES")
                elif entity1_type == "Application" and entity2_type == "Technique":
                    session.write_transaction(create_relationship, entity1_name, entity2_name, "IMPLEMENTED_WITH")
                elif entity1_type == "Game" and entity2_type == "Concept":
                    session.write_transaction(create_relationship, entity1_name, entity2_name, "BASED_ON")
                elif entity1_type == "Field" and entity2_type == "Concept":
                    session.write_transaction(create_relationship, entity1_name, entity2_name, "STUDIES")
                elif entity1_type == "Field" and entity2_type == "Goal":
                    session.write_transaction(create_relationship, entity1_name, entity2_name, "SUPPORTS")
                elif entity1_type == "Concept" and entity2_type == "Field":
                    session.write_transaction(create_relationship, entity1_name, entity2_name, "APPLIES_TO")
                elif entity1_type == "Year" and entity2_type == "Concept":
                    session.write_transaction(create_relationship, entity1_name, entity2_name, "INTRODUCED_IN")
                elif entity1_type == "Goal" and entity2_type == "Technique":
                    session.write_transaction(create_relationship, entity1_name, entity2_name, "ACHIEVED_BY")
                elif entity1_type == "Concept" and entity2_type == "Technique":
                    session.write_transaction(create_relationship, entity1_name, entity2_name, "ENABLED_BY")


<ipython-input-14-39c5d64e944f>:5: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_node, entity1_type, entity1_name)
<ipython-input-14-39c5d64e944f>:23: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_relationship, entity1_name, entity2_name, "APPLIES_TO")
<ipython-input-14-39c5d64e944f>:29: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_relationship, entity1_name, entity2_name, "ENABLED_BY")
<ipython-input-14-39c5d64e944f>:19: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_relationship, entity1_name, entity2_name, "STUDIES")
<ipython-input-14-39c5d64e944f>:21: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_relationship, entity1_name, entity2_name, "SUPPORTS")
<ipython-input-14-39c5d64e944f>:1